In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import pandas as pd
from datetime import datetime, timedelta
import autogluon.core as ag
import matplotlib.pyplot as plt

# Data

In [ ]:
df = pd.read_csv('/kaggle/input/agtech-ai-hackathon-online-2023/train.csv')
df = df[:1095]

In [ ]:
# Assuming your data is in a pandas DataFrame named 'df'
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

In [ ]:
df.isnull().sum()

In [ ]:
# fill missing values with the mean
mean_value = df['PM2.5'].mean()
print(f'mean_value : {mean_value}')

df = df.fillna({'PM2.5': mean_value})

# Check again if there are still NaN values in the data
print(df.isnull().sum())

In [ ]:
new = pd.DataFrame(['PM2.5']*df.shape[0])
df_format = pd.concat([df,new],axis=1)

In [ ]:
df_format

In [ ]:
# Create a TimeSeriesDataFrame
train_data = TimeSeriesDataFrame.from_data_frame(
    df_format,
    id_column=0,
    timestamp_column='Date'
)

In [ ]:
train_data

In [ ]:
train_data = train_data.to_regular_index(freq="D")

In [ ]:
train_data

# Autogluon

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=366,
    path="Model",
    target="PM2.5",
    eval_metric="RMSE"
)

predictor.fit(
    train_data,
    presets = "best_quality"
)

In [ ]:
predictions = predictor.predict(train_data)

In [ ]:
predictions

# Plot Graph

In [ ]:
# TimeSeriesDataFrame can also be loaded directly from a file
plt.figure(figsize=(20, 3))

item_id = "PM2.5"
y_past = train_data.loc[item_id]["PM2.5"]
y_pred = predictions.loc[item_id]
# y_test = test_data.loc[item_id]["PM2.5"][-48:]

plt.plot(y_past[-200:], label="Past time series values")
plt.plot(y_pred["mean"], label="Mean forecast")
# plt.plot(y_test, label="Future time series values")

plt.fill_between(
    y_pred.index, y_pred["0.1"], y_pred["0.9"], color="red", alpha=0.1, label=f"10%-90% confidence interval")
plt.legend();

# Submission

In [ ]:
# Define start date and number of days
start_date = datetime(2016, 1, 1)
num_days = 366

# Create a list of dates using pandas date_range function
date_range = pd.date_range(start=start_date, periods=num_days, freq='D')

In [ ]:
# create a new DataFrame with the dates and predictions
prediction_df = pd.DataFrame({
    'Date': date_range ,
    'PM2.5': predictions['mean']
})

# format the 'Date' column to 'D/M/Y' pattern
prediction_df['Date'] = prediction_df['Date'].dt.strftime('%-d/%-m/%Y')

In [ ]:
# save the DataFrame to a CSV file
prediction_df.to_csv('BestW-En-mean.csv', index=False)